# PDF Text Extractor Utility

> Extracts unstructured text from scientific papers published as PDF files .

In [1]:
#| default_exp utils.pdf_research_article_text_extractor

In [5]:
#| export
import dataclasses
from langchain.document_loaders.pdf import BasePDFLoader
from langchain.document_loaders.base import BaseBlobParser
from langchain.docstore.document import Document
from langchain.document_loaders.blob_loaders import Blob
import nltk
nltk.download('punkt')
import nltk.data
import re
from typing import Optional, List, Iterator, Mapping, Any, Dict

@dataclasses.dataclass
class PyMuPDFBlock:
    """A block of text occuring in a PDF file."""

    page: int

    x0:float
    x1:float
    width:float

    y0:float
    y1:float
    height:float
    
    nlines:int
    font_size_proxy:float

    text:str
    t:str

    def __init__(self, p:int, x0:float, y0:float, x1:float, y1:float, text:str):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1
        self.page = p
        
        self.text = text
        self.width = x1 - x0
        self.height = y1 - y0
        self.nlines = text.count('\n')
        if self.nlines == 0:
            self.nlines = 1 
        self.font_size_proxy = (self.height / self.nlines)
        self.t = re.sub('\-\n','',self.text)
        self.t = re.sub('\n',' ',self.text)
        self.t = re.sub('\s+',' ',self.text)
        
    def __repr__(self):
        return f"PyMuPDFBlock(page={self.page}, x0={self.x0}, x1={self.x1}, y0={self.y0}, y1={self.y1}, text={self.text})"
    
    def __str__(self): 
        return f"PyMuPDFBlock(page={self.page}, x0={self.x0}, x1={self.x1}, y0={self.y0}, y1={self.y1}, text={self.text})"
    
    def __eq__(self, other):
        if self.page == other.page and self.x0 == other.x0 and self.x1 == other.x1 and self.y0 == other.y0 and self.y1 == other.y1:
            return True
        else:
            return False

class PyMuPDFBlockLoader(BasePDFLoader):
    """Load `PDF` files using `PyMuPDF`."""

    def __init__(self, file_path: str) -> None:
        """Initialize with a file path."""
        try:
            import fitz  # noqa:F401
        except ImportError:
            raise ImportError(
                "`PyMuPDF` package not found, please install it with "
                "`pip install pymupdf`"
            )

        super().__init__(file_path)

    def load(self, **kwargs: Optional[Any]) -> List[Document]:
        """Load file."""

        parser = PyMuPDFBlockParser(text_kwargs=kwargs)
        blob = Blob.from_path(self.file_path)
        blocks = parser.parse(blob)
    
        return blocks
    
class PyMuPDFBlockParser(BaseBlobParser):
    """Parse `PDF` using `PyMuPDF`."""

    def __init__(self, text_kwargs: Optional[Mapping[str, Any]] = None) -> None:
        """Initialize the parser.

        Args:
            text_kwargs: Keyword arguments to pass to ``fitz.Page.get_text()``.
        """
        self.text_kwargs = text_kwargs or {}

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Lazily parse the blob."""
        import fitz
        sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
 
        with blob.as_bytes_io() as file_path:

            doc = fitz.open(file_path)  # open document

            text_dict = {}
            sdist = {}
            fdist = {}        
            all_spans = []
            for page in doc:
                d = page.get_textpage().extractDICT()
                for block in d.get('blocks'):
                    for line in block.get('lines'):
                        for span in line.get('spans'):
                            all_spans.append(span)
                            text = span.get('text','')
                            text_size = span.get('size',0)
                            if sdist.get(text_size) is None:
                                sdist[text_size] = len(text)
                            else:
                                sdist[text_size] += len(text)                    
                            text_font = span.get('font',0)
                            if fdist.get(text_font) is None:
                                fdist[text_font] = len(text)
                            else:
                                fdist[text_font] += len(text)
            
            # VERY SIMPLE APPROACH TO START WITH
            # Find the most common font size and use that to estimate the 
            # bounds of the page to exclude headers and footers
            #
            # Want to add a structured model to predict type of 
            # text blocks in the document  

            most_common_font = max(sdist, key=sdist.get)

            min_x0 = min([sp.get('bbox')[0]  for sp in all_spans if sp.get('size') == most_common_font])
            min_y0 = min([sp.get('bbox')[1]  for sp in all_spans if sp.get('size') == most_common_font])
            max_x1 = max([sp.get('bbox')[2]  for sp in all_spans if sp.get('size') == most_common_font])
            max_y2 = max([sp.get('bbox')[3]  for sp in all_spans if sp.get('size') == most_common_font])
                        
            doc_text = ''
            for page in doc:
                for (x0, y0, x1, y1, text, bn, bt) in page.get_textpage().extractBLOCKS():
                    if x0 < min_x0 or y0 < min_y0 or x1 > max_x1 or y1 > max_y2:
                        continue
                    doc_text += text
                    doc_text += '\n\n'        
            yield Document(
                    page_content=doc_text,
                    metadata=dict({
                        "file_path": blob.source,
                        "total_pages": len(doc),
                    }))


[nltk_data] Downloading package punkt to /Users/gburns/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
